# Detecting Whether a Shot Has Happened

I found out that it is very costly to run detectron keypoint detector and instance predictor on every single frame of a video.
Even with GPU acceleration, processing one second of a video takes around (24 frames) x 1 second = 24 seconds

Thus, it is important to identify the parts of the video where a shot is happening first.

Inspired by: https://www.youtube.com/watch?v=QKVpIo5sfGA

In [3]:
import skvideo.io
import skvideo.datasets

import cv2

In [8]:
def get_frame_difference(frame, prev_frames = []):
    """
        Returns the current
    """
    
    
    
    pass

In [10]:
cap = cv2.VideoCapture(0)
ret, current_frame = cap.read()
previous_frame = current_frame

while(cap.isOpened()):
    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    previous_frame_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)    

    frame_diff = cv2.absdiff(current_frame_gray, previous_frame_gray)

    cv2.imshow('frame diff ',frame_diff)      
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    previous_frame = current_frame.copy()
    ret, current_frame = cap.read()

cap.release()
cv2.destroyAllWindows()